# Testing QA4MBES functionality 2: grid coverage testing


In [1]:
from shapely.geometry import shape
import json

In [2]:
## awkward hack to import local modules in Jupyter
import sys
sys.executable
sys.path.append('/mnt/bigdata/frontierSI/qa4mbes-data-pipeline/qa4mbes')

In [3]:
import testcoverage
import getpointcoverage

## 1. Check geotiff grid coverage of a planned area

Does my geotiff raster cover any of the planned survey region? if so how much? if not roughly how far away were we?

### The positive case - we surveyed some of the planned region

In [ ]:
%%time

coveragestats = testcoverage.testcoverage('/mnt/bigdata/frontierSI/sampledata/raster/grid1_1m-crs.tiff', '../tests/testjson.geojson')

In [5]:
coveragestats

{'QAfailed': 'No CRS present',
 'filename': '/mnt/bigdata/frontierSI/sampledata/raster/grid2_1m.tif'}

In [6]:
from testcoverage import jsontoshapely
jsontoshapely(coveragestats["intersection"])

KeyError: 'intersection'

### The null case - we didn't survey any of the planned region

In [ ]:
nullstats = testcoverage.testcoverage('../tests/4819-100000lines.xyz', '../tests/nulltest.shp')

In [ ]:
nullstats

### Interpreting results

`testcoverage` returns a python dictionary containing:

- time of test start
- time of test stop
- path to the 'planned' coverage
- path to the survey swathe being tested
- % of planned coverage overlapped by the swath
- area (in metres) of the planned coverage overlapped by the swath
- distance (in metres) between the centroids of the planned coverage and the swath coverage
- minimum distance (in metres) between the planned and survey coverages. This is a rough guide mainly used to see how far away the survey was in the case of no overlap with the planned region
- a GeoJSON polygon describing the intersection of planned coverage and survey coverage

Optionally, it could contain GeoJSON polygons describing the actual coverages used in the test.

**question:** should swath density metrics be included here?

## 2. Check BAG coverage of a planned area

Does my bathymetry attributed grid cover any of the planned survey region? if so how much? if not roughly how far away were we?

In [198]:
# handling BAG
import h5py
from io import BytesIO
from lxml import etree


In [199]:
bagfile = h5py.File("../../bag-samples/grid1_ellipsoid_1m.bag")

In [200]:
for item in bagfile["BAG_root"].items():
    print(item)

('elevation', <HDF5 dataset "elevation": shape (6551, 6501), type "<f4">)
('metadata', <HDF5 dataset "metadata": shape (47164,), type "|S1">)
('tracking_list', <HDF5 dataset "tracking_list": shape (0,), type "|V20">)
('uncertainty', <HDF5 dataset "uncertainty": shape (6551, 6501), type "<f4">)


In [201]:
root = bagfile['BAG_root']

In [203]:
metadata = root["metadata"]

In [208]:
metadata.value

array([b'<', b'?', b'x', ..., b'>', b'\n', b''], dtype='|S1')

In [209]:
buffer = BytesIO(metadatanode.value)
metadatatree = etree.parse(buffer)
mroot = metadatatree.getroot()

In [196]:
#print(etree.tostring(root, pretty_print=True).decode('ascii'))

In [215]:
for item in mroot:
    print(item.tag)

{http://www.isotc211.org/2005/gmd}fileIdentifier
{http://www.isotc211.org/2005/gmd}language
{http://www.isotc211.org/2005/gmd}characterSet
{http://www.isotc211.org/2005/gmd}hierarchyLevel
{http://www.isotc211.org/2005/gmd}contact
{http://www.isotc211.org/2005/gmd}dateStamp
{http://www.isotc211.org/2005/gmd}metadataStandardName
{http://www.isotc211.org/2005/gmd}metadataStandardVersion
{http://www.isotc211.org/2005/gmd}spatialRepresentationInfo
{http://www.isotc211.org/2005/gmd}spatialRepresentationInfo
{http://www.isotc211.org/2005/gmd}referenceSystemInfo
{http://www.isotc211.org/2005/gmd}referenceSystemInfo
{http://www.isotc211.org/2005/gmd}identificationInfo
{http://www.isotc211.org/2005/gmd}distributionInfo
{http://www.isotc211.org/2005/gmd}dataQualityInfo
{http://www.isotc211.org/2005/gmd}metadataConstraints
{http://www.isotc211.org/2005/gmd}metadataConstraints
{http://www.isotc211.org/2005/gmi}acquisitionInformation


In [229]:
mroot.nsmap

{'gmi': 'http://www.isotc211.org/2005/gmi',
 'gmd': 'http://www.isotc211.org/2005/gmd',
 'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 'gml': 'http://www.opengis.net/gml/3.2',
 'gco': 'http://www.isotc211.org/2005/gco',
 'xlink': 'http://www.w3.org/1999/xlink',
 'bag': 'http://www.opennavsurf.org/schema/bag'}

In [234]:
mroot.get('MD_ReferenceSystem')

In [228]:
mroot.findtext('MD_ReferenceSystem')

In [164]:

sri = etree.QName(metadataroot.nsmap['gmd'], 'spatialRepresentationInfo').text
mdr = etree.QName(metadataroot.nsmap['gmd'], 'MD_ReferenceSystem').text
ref = etree.QName(metadataroot.nsmap['gmd'], 'referenceSystemIdentifier').text
code = etree.QName(metadataroot.nsmap['gmd'], 'code').text
crs= etree.QName(metadataroot.nsmap['gco'], 'CharacterString').text



In [171]:
resolution = (
    root
    .find('.//{}'.format(sri))
    .find('.//{}'.format(mdr))
    .find('.//{}'.format(ref))
    .find('.//{}'.format(code))
    .find('.//{}'.format(crs))
)
print(resolution.text, resolution.get('crs'))



AttributeError: 'NoneType' object has no attribute 'find'

In [174]:
root.nsmap

{'gmi': 'http://www.isotc211.org/2005/gmi',
 'gmd': 'http://www.isotc211.org/2005/gmd',
 'xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 'gml': 'http://www.opengis.net/gml/3.2',
 'gco': 'http://www.isotc211.org/2005/gco',
 'xlink': 'http://www.w3.org/1999/xlink',
 'bag': 'http://www.opennavsurf.org/schema/bag'}

In [195]:
sri = etree.QName(root.nsmap['gmd'], 'spatialRepresentationInfo').text
adp = etree.QName(root.nsmap['gmd'], 'axisDimensionProperties').text
dim = etree.QName(root.nsmap['gmd'], 'MD_Dimension').text
res = etree.QName(root.nsmap['gmd'], 'resolution').text
res_meas = etree.QName(root.nsmap['gco'], 'Measure').text

srs = etree.QName(root.nsmap['gco'], 'CharacterString').text

ValueError: Invalid tag name 'http://www.isotc211.org/2005/gmd'

In [187]:
resolution = root.find('.//{}'.format(res_meas))
print(resolution.text, resolution.get('uom'))

1.000000000000 m


In [194]:
root.nsmap['gmd'][1]

't'

In [ ]:
root.find('.//{}'.format(srs)).text

In [51]:
south_bound_lat = etree.QName(metadataroot.nsmap['gmd'], 'southBoundLatitude').text

In [55]:
metadataroot.find('.//{}'.format(south_bound_lat))

<Element {http://www.isotc211.org/2005/gmd}southBoundLatitude at 0x7f46c0596ac8>

In [61]:
metadataroot.attrib

{}

## 4. Test grid density
...is my 1m grid really a 1m grid?